## This notebooks checks the different parameters obtained in each experiment file

In [1]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from scipy.signal import freqs,periodogram,cheby1
import numpy as np

# import quantum libraries
import qutip
from itertools import product
from numpy import array, kron
from qmldataset import pauli_operators, create_custom_simulator, run_experiment

2021-09-25 18:12:19.245511: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [2]:
dimension = 4
evolution_time = 1
num_time_steps = 1024
omega = [10, 12]
dynamic_operators = [
    kron(pauli_operators[1], pauli_operators[0]),
    kron(pauli_operators[0], pauli_operators[1]),
    kron(pauli_operators[1], pauli_operators[1])]
static_operators = [
    omega[0] * kron(pauli_operators[3], pauli_operators[0]),
    omega[1] * kron(pauli_operators[0], pauli_operators[3])]
noise_operators = [
    kron(pauli_operators[3], pauli_operators[0]),
    kron(pauli_operators[0], pauli_operators[3])]
measurement_operators = [
    kron(meas_op_one, meas_op_two) for meas_op_one, meas_op_two in list(
        product(pauli_operators, pauli_operators))][1:]
initial_states_1q = [
    array([[0.5, 0.5], [0.5, 0.5]]), array([[0.5, -0.5], [-0.5, 0.5]]),
    array([[0.5, -0.5j], [0.5j, 0.5]]), array([[0.5, 0.5j], [-0.5j, 0.5]]),
    array([[1, 0], [0, 0]]), array([[0, 0], [0, 1]])
]
initial_states = [
    kron(init_state_one, init_state_two) for init_state_one, init_state_two in list(
        product(initial_states_1q, initial_states_1q))]
num_realizations = 500
num_pulses = 5
noise_profile = ['Type 1', 'Type 5']

simulator = create_custom_simulator(
    evolution_time=evolution_time,
    num_time_steps=num_time_steps,
    dimension=dimension,
    dynamic_operators=dynamic_operators,
    static_operators=static_operators,
    noise_operators=noise_operators,
    measurement_operators=measurement_operators,
    initial_states=initial_states,
    num_realizations=num_realizations,
    num_pulses=num_pulses,
    noise_profile=noise_profile,
    distortion=True,
    pulse_shape="Square"
)

2021-09-25 18:12:24.330469: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-09-25 18:12:24.331756: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-09-25 18:12:24.385209: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-25 18:12:24.385663: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:0e:00.0 name: GeForce GTX 1050 Ti computeCapability: 6.1
coreClock: 1.43GHz coreCount: 6 deviceMemorySize: 3.94GiB deviceMemoryBandwidth: 104.43GiB/s
2021-09-25 18:12:24.385710: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-09-25 18:12:24.389112: I tensorflow/stream_executor/platform/default/

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
signal_generator (SignalGenerat ((None, 5, 3), (None 0           input_1[0][0]                    
__________________________________________________________________________________________________
signal_generator_1 (SignalGener ((None, 5, 3), (None 0           input_1[0][0]                    
__________________________________________________________________________________________________
signal_generator_2 (SignalGener ((None, 5, 3), (None 0           input_1[0][0]                    
______________________________________________________________________________________________

__________________________________________________________________________________________________
rho30M14 (QuantumMeasurement)   (None, 500, 1)       0           V14[0][0]                        
                                                                 Uc[0][0]                         
__________________________________________________________________________________________________
rho31M0 (QuantumMeasurement)    (None, 500, 1)       0           V0[0][0]                         
                                                                 Uc[0][0]                         
__________________________________________________________________________________________________
rho31M1 (QuantumMeasurement)    (None, 500, 1)       0           V1[0][0]                         
                                                                 Uc[0][0]                         
__________________________________________________________________________________________________
rho31M2 (Q

In [3]:
experiment_result = run_experiment(simulator=simulator)

2021-09-25 18:12:52.583931: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-09-25 18:12:52.601030: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 3094165000 Hz
2021-09-25 18:13:35.077395: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2021-09-25 18:13:35.475934: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.11
2021-09-25 18:13:35.632747: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcufft.so.10
2021-09-25 18:13:35.949198: I tensorflow/core/util/cuda_solvers.cc:180] Creating CudaSolver handles for stream 0x559bad802500
2021-09-25 18:13:35.949443: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcusolver.so.10
2021-09-25 18:13:36.276

1/1 [==============================] - 47s 47s/step


In [4]:
# Retrieve the parameters
simulation_params = experiment_result["sim_parameters"]
for param in simulation_params:
    print('-- {} --'.format(param))
    print(simulation_params[param])

-- evolution_time --
1
-- num_time_steps --
1024
-- dynamic_operators --
[array([[0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [1., 0., 0., 0.],
       [0., 1., 0., 0.]]), array([[0., 1., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 1., 0.]]), array([[0., 0., 0., 1.],
       [0., 0., 1., 0.],
       [0., 1., 0., 0.],
       [1., 0., 0., 0.]])]
-- static_operators --
[array([[ 10.,   0.,   0.,   0.],
       [  0.,  10.,   0.,   0.],
       [  0.,   0., -10.,  -0.],
       [  0.,   0.,  -0., -10.]]), array([[ 12.,   0.,   0.,   0.],
       [  0., -12.,   0.,  -0.],
       [  0.,   0.,  12.,   0.],
       [  0.,  -0.,   0., -12.]])]
-- noise_operators --
[array([[ 1.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.],
       [ 0.,  0., -1., -0.],
       [ 0.,  0., -0., -1.]]), array([[ 1.,  0.,  0.,  0.],
       [ 0., -1.,  0., -0.],
       [ 0.,  0.,  1.,  0.],
       [ 0., -0.,  0., -1.]])]
-- measurement_operators --
[array([[0., 1., 0., 0.],
       [1., 0.,

In [5]:
# pulse parameter shape is [num_pulses, len(dynamic_operators)x3]
# for each pulse, the 3 parameters are amplitude, position, width
print("Pulse parameters shape {}".format(experiment_result["pulse_parameters"].shape))
print("Pulse Parameters value {} ".format(experiment_result["pulse_parameters"]))


Pulse parameters shape (5, 9)
Pulse Parameters value [[-89.00962      0.12295276   0.1        -28.82912      0.11125359
    0.1        -18.901085     0.11355914   0.1       ]
 [ 83.00755      0.2774218    0.1         57.943108     0.2631473
    0.1         15.187311     0.28344297   0.1       ]
 [  8.973074     0.4530517    0.1         60.60338      0.44219956
    0.1         79.512695     0.45212716   0.1       ]
 [-67.06779      0.6068499    0.1        -88.53671      0.6192637
    0.1        -71.219345     0.6243191    0.1       ]
 [-70.189575     0.7699057    0.1         63.615395     0.786229
    0.1        -67.32652      0.7789102    0.1       ]] 


In [6]:
# both pulse and distorted_pulses contain actual values of the pulse
# shape is [time_steps, 1, len(dynamic_operators)x3]
print("Pulses shape {}".format(experiment_result["pulses"].shape))
print("Distorted pulses shape {} ".format(experiment_result["distorted_pulses"].shape))

Pulses shape (1024, 1, 3)
Distorted pulses shape (1024, 1, 3) 


In [7]:
# Noise shape (time_steps, num_realizations, len(noise_profile))
# For each profile, it contains the value of noise in each realization
print("Noise shape {}".format(experiment_result["noise"].shape))


Noise shape (1024, 500, 2)


In [8]:
# Expectation shape is (len(measurement_operators)xlen(initial_states))
# Each value is the observable of each initial state
print("Expectations shape {}".format(experiment_result["expectations"].shape))

# Average expectation is the average of expectations (over num_realizations)
# each element is the average of expectations over number of 
# realizations

print(experiment_result["average_expectation"].shape)

Expectations shape (500, 540)
(540,)


In [9]:
# H0 (control) shape is (time_steps, 1, dimension, dimension)
print("H0 shape {}".format(experiment_result["H0"].shape))
# H1 (noise) shape is (time_steps, num_realization, dimension, dimension)
print("H1 shape {}".format(experiment_result["H1"].shape))

H0 shape (1024, 1, 4, 4)
H1 shape (1024, 500, 4, 4)


In [10]:
# U0 (control) shape is (time_steps, 1, dimension, dimension)
# Unitary values at each time steps
print("U0 shape{}".format(experiment_result["U0"].shape))
# UI (noise) shape is (num_realizations, dimension, dimension)
# Total unitary at evolution time for each noise realization
print("UI shape{}".format(experiment_result["UI"].shape))

U0 shape(1024, 1, 4, 4)
UI shape(500, 4, 4)


In [11]:
# Vo operator is a length equal to len(measurement_operators)
# Each operator is of shape (dimesion, dimension)
print("Vo operator shape {}".format(len(experiment_result["average_vo"])))
print("Single Vo operator shape {}".format(experiment_result["average_vo"][0].shape))

# V0 (measurements) is a list of length equal to len(measurement_operators)
print("Vo shape {}".format(len(experiment_result["vo"])))
# Each element of Vo is of shape (num_realization, dimension, dimension)
print("Single Vo {}".format(experiment_result["vo"][0].shape))


print(np.average(experiment_result["vo"][0], axis=0))
print(experiment_result["average_vo"][0])

Vo operator shape 15
Single Vo operator shape (4, 4)
Vo shape 15
Single Vo (500, 4, 4)
[[ 0.5786319 +8.4137835e-02j -0.07135502-2.6703403e-10j
   0.0486394 -4.8702038e-04j -0.02747885+2.4300443e-02j]
 [ 0.07381808-3.7768799e-10j  0.5786318 -8.4137842e-02j
  -0.01936717-2.9075701e-02j  0.05754664+2.9806510e-02j]
 [ 0.05754664-2.9806506e-02j -0.02747885-2.4300436e-02j
   0.6391467 +2.0562207e-02j  0.04693115+2.1576092e-10j]
 [-0.01936717+2.9075706e-02j  0.0486394 +4.8702123e-04j
  -0.04938913-3.6037479e-10j  0.63914675-2.0562204e-02j]]
[[ 0.5786319 +8.4137835e-02j -0.07135502-2.6703403e-10j
   0.0486394 -4.8702038e-04j -0.02747885+2.4300443e-02j]
 [ 0.07381808-3.7768799e-10j  0.5786318 -8.4137842e-02j
  -0.01936717-2.9075701e-02j  0.05754664+2.9806510e-02j]
 [ 0.05754664-2.9806506e-02j -0.02747885-2.4300436e-02j
   0.6391467 +2.0562207e-02j  0.04693115+2.1576092e-10j]
 [-0.01936717+2.9075706e-02j  0.0486394 +4.8702123e-04j
  -0.04938913-3.6037479e-10j  0.63914675-2.0562204e-02j]]
